In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark.mllib.util import MLUtils

%pylab inline

import numpy as np
from time import time

import sys,os,pickle
sys.path.insert(0, os.path.abspath('../src'))

from BoostStumps import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pyspark import SparkContext

#sc=SparkContext()

from numpy.random import rand
p=0.9
data=[]
for i in range(10):
    for j in range(10):
        if np.abs(i-4)<3 and np.abs(j-6)<3:
            y=2*(rand()<p)-1
        else:
            y=2*(rand()>p)-1
        print "%1.0f "%((1+y)/2),
        data.append(LabeledPoint(y,[i,j]))
    print

0  0  0  0  1  0  0  0  0  0 
0  1  0  0  0  0  0  0  0  0 
0  0  0  0  1  1  1  1  1  0 
0  0  0  0  1  1  1  1  1  0 
0  0  0  0  1  1  0  1  1  0 
0  0  0  0  1  1  1  1  1  0 
0  1  0  0  1  1  1  1  1  0 
0  0  0  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  1  0 
0  0  0  0  0  0  0  0  0  0 


In [21]:
dataRDD=sc.parallelize(data,numSlices=2)
dataRDD.getNumPartitions()

dataRDD = sc.textFile("../data/covtype.data").map(lambda line: [float(x.strip()) for x in line.split(',')]).map(lambda line: LabeledPoint(line[-1],line[:-1])).cache()
#dataRDD = dataRDD.sample(False,0.1, seed=255).cache()
dataRDD.first()

LabeledPoint(5.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [22]:
sc

In [23]:
booster=Booster(sc,dataRDD)

number of features= 54 number of partitions= 54
Sizes: Data1=581012, trainingData=406538, testData=174474
number of elements in GR= 54
number of elements in GTR= 54
number of partitions in PS= 54


In [24]:
print booster.report_times()

 17.88 : Split into train and test
  0.00 : glom
  0.00 : add partition index
 10.22 : Prepare_partition_data_structure
  0.45 : Compute Split points
  1.29 : Add_weak_learner_matrix
  0.00 : Finished Initialization


In [27]:
Scores=[]
for i in range(10):
    booster.boosting_iteration()
    Scores.append(booster.compute_scores())

In [28]:
print booster.report_times()

 17.88 : Split into train and test
  0.00 : glom
  0.00 : add partition index
 10.22 : Prepare_partition_data_structure
  0.45 : Compute Split points
  1.29 : Add_weak_learner_matrix
  0.00 : Finished Initialization
 16.12 : Start main loop 0
  0.56 : found best splitter 0
  1.47 : Updated Weights 0
  0.00 : Start main loop 1
  0.53 : found best splitter 1
  1.66 : Updated Weights 1
  0.00 : Start main loop 2
  0.51 : found best splitter 2
  1.67 : Updated Weights 2
  0.00 : Start main loop 3
  0.63 : found best splitter 3
  1.47 : Updated Weights 3
  0.00 : Start main loop 4
  0.57 : found best splitter 4
  1.54 : Updated Weights 4
  0.00 : Start main loop 5
  0.70 : found best splitter 5
  1.49 : Updated Weights 5
  0.00 : Start main loop 6
  0.64 : found best splitter 6
  1.81 : Updated Weights 6
  0.00 : Start main loop 7
  0.59 : found best splitter 7
  1.53 : Updated Weights 7
  0.00 : Start main loop 8
  0.87 : found best splitter 8
  1.63 : Updated Weights 8
  0.00 : Start main

In [29]:
booster.proposals

[{'best feature': 53,
  'details': [{'Correlation': 2.037839968466693,
    'Feature_index': 0,
    'SS': array([-1.33070556, -0.85218762, -0.45828406, -0.08881881,  0.25515701,
            0.58468007,  0.85823151,  1.13204572,  1.48443043,  2.03783997]),
    'Threshold': 1.7976931348623157e+308,
    'Threshold_index': 9},
   {'Correlation': 2.109919215997881,
    'Feature_index': 1,
    'SS': array([-1.72109654, -1.33995497, -0.90213217, -0.49821216, -0.04370282,
            0.39147133,  0.81578599,  1.23109522,  1.6575288 ,  2.10991922]),
    'Threshold': 1.7976931348623157e+308,
    'Threshold_index': 9},
   {'Correlation': 2.091163162097418,
    'Feature_index': 2,
    'SS': array([-1.74434389, -1.40710673, -0.96712803, -0.69776417, -0.22717594,
            0.17527282,  0.54737823,  1.03287197,  1.48882087,  2.09116316]),
    'Threshold': 1.7976931348623157e+308,
    'Threshold_index': 9},
   {'Correlation': 2.0321219545392677,
    'Feature_index': 3,
    'SS': array([-1.84646795, -